In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import random
import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import rcParams
%matplotlib inline
rcParams['figure.figsize'] = 10,8
sns.set(style='whitegrid', palette='muted',
        rc={'figure.figsize': (15,10)})
import os
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout

from numpy.random import seed
from tensorflow import set_random_seed

Using TensorFlow backend.


In [2]:
train_dataset = pd.read_csv("energy_train.csv")
test_dataset = pd.read_csv("energy_test.csv")

train_dataset = train_dataset.set_index("id")
test_dataset = test_dataset.set_index("id")
train_dataset.dropna(axis=0,  inplace=True)


train_dataset.columns = ['Date','WattHour', 'Lumin','D1','M1','D2','M2','D3','M3','D4','M4','D5','M5','D6','M6','D7','M7','D8','M8','D9','M9','D_Cout','Pressure','M_Cout','Wind','Clarity','Dew','R1','R2']

test_dataset.columns = ['Date', 'Lumin','D1','M1','D2','M2','D3','M3','D4','M4','D5','M5','D6','M6','D7','M7','D8','M8','D9','M9','D_Cout','Pressure','M_Cout','Wind','Clarity','Dew','R1','R2']



train_dataset.dropna(axis=0,inplace=True)
train_dataset['Lumin'] = train_dataset['Lumin'].astype(float)
test_dataset['Lumin'] = test_dataset['Lumin'].astype(float)
train_dataset.round(decimals=2)
test_dataset.round(decimals=2)
np.nan_to_num(train_dataset)
np.nan_to_num(test_dataset)
train_dataset.fillna("bfill")
test_dataset.fillna("bfill")
test_dataset

,Date,Lumin,D1,M1,D2,M2,D3,M3,D4,M4,...,D9,M9,D_Cout,Pressure,M_Cout,Wind,Clarity,Dew,R1,R2
id,,,,,,,,,,,,,,,,,,,,,
15001,2016-04-24 21:00:00,0.0,21.926667,35.500000,19.290000,37.500000,22.390000,34.000000,21.390000,32.225714,...,20.200000,NaN,4.100000,758.000000,82.000000,3.0,40.000000,1.200000,10.668196,10.668196
15002,2016-04-24 21:10:00,0.0,21.890000,35.400000,19.222500,37.425000,22.390000,34.090000,21.350000,32.200000,...,20.200000,32.400000,3.950000,758.050000,82.166667,3.0,40.000000,1.100000,48.467852,48.467852
15003,2016-04-24 21:20:00,0.0,21.890000,35.400000,19.200000,37.466667,22.390000,33.963333,21.290000,32.277143,...,20.200000,32.290000,3.800000,758.100000,82.333333,3.0,40.000000,1.000000,36.388536,36.388536
15004,2016-04-24 21:30:00,0.0,21.890000,35.400000,19.100000,37.590000,22.390000,33.900000,21.290000,32.334000,...,20.175000,32.290000,3.650000,758.150000,82.500000,3.0,40.000000,0.900000,17.198176,17.198176
15005,2016-04-24 21:40:00,0.0,21.890000,35.400000,19.100000,37.590000,22.390000,33.966667,21.290000,32.290000,...,20.166667,32.563333,3.500000,758.200000,82.666667,3.0,40.000000,0.800000,7.200588,7.200588
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19371,2016-05-25 05:20:00,10.0,23.600000,40.833333,20.856667,43.566667,25.260000,39.266667,23.500000,39.000000,...,22.600000,44.790000,6.166667,757.133333,95.000000,1.0,24.666667,5.433333,43.792662,43.792662
19372,2016-05-25 05:30:00,0.0,23.666667,40.566667,20.790000,43.700000,25.133333,38.666667,23.500000,38.933333,...,22.600000,44.760000,6.150000,757.100000,95.500000,1.0,24.500000,5.500000,22.151890,22.151890
19373,2016-05-25 05:40:00,0.0,23.600000,40.093333,20.700000,43.296667,24.963333,37.833333,23.463333,38.563333,...,22.600000,44.760000,6.133333,757.066667,96.000000,1.0,24.333333,5.566667,5.759009,5.759009


In [3]:
X = train_dataset.drop(['WattHour','Date'], axis=1)
y = train_dataset["WattHour"]

In [4]:

from sklearn.model_selection import train_test_split
# here I used 20% data for test_data and 80% as train_data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [5]:
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import make_scorer, r2_score, mean_squared_log_error

n_folds = 5

cv = KFold(n_splits = 5, shuffle=True, random_state=42).get_n_splits(x_train.values)

def test_model(model):   
    msle = make_scorer(mean_squared_log_error)
    rmsle = np.sqrt(cross_val_score(model, x_train, y_train, cv=cv, scoring = msle))
    score_rmsle = [rmsle.mean()]
    return score_rmsle

def test_model_r2(model):
    r2 = make_scorer(r2_score)
    r2_error = cross_val_score(model, x_train, y_train, cv=cv, scoring = r2)
    score_r2 = [r2_error.mean()]
    return score_r2

In [6]:
from sklearn.linear_model import ElasticNet

reg_elastic_net = ElasticNet(alpha=0.1, copy_X=True, fit_intercept=True, l1_ratio=0.9,
           max_iter=100, normalize=False, positive=False, precompute=False,
           random_state=None, selection='cyclic', tol=0.0001, warm_start=False)

#clf_elastic_net.get_params()
rmsle_elastic_net = test_model(reg_elastic_net)

print (rmsle_elastic_net, test_model_r2(reg_elastic_net))

[0.639580242458176] [0.16078673404538116]


C:\Users\Aboli\Anaconda3\envs\Intern1\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7815272.881006017, tolerance: 4327.34338387485
  positive)
C:\Users\Aboli\Anaconda3\envs\Intern1\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 7949507.535613837, tolerance: 4651.908606498196
  positive)
C:\Users\Aboli\Anaconda3\envs\Intern1\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 10957866.335872814, tolerance: 4469.369780986763
  positive)
C:\Users\Aboli\Anaconda3\envs\Intern1\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might 

In [7]:
from sklearn import svm
clf_svm = svm.SVR()
rmsle_svm = test_model(clf_svm)
print (rmsle_svm, test_model_r2(clf_svm))

[0.6815714362244631] [-0.11661489468730761]


In [8]:
from sklearn.tree import DecisionTreeRegressor
clf_dtR = DecisionTreeRegressor(max_depth=5, random_state=51)
rmsle_dtR = test_model(clf_dtR)
print (rmsle_dtR, test_model_r2(clf_dtR))

[0.6365377861715926] [0.06581893933730294]


In [9]:
from sklearn.ensemble import RandomForestRegressor

clf_rFR = RandomForestRegressor(max_depth=5, random_state=51)
rmsle_rFR = test_model(clf_rFR)
print (rmsle_rFR, test_model_r2(clf_rFR))

[0.6129868054572698] [0.1652678961610317]


In [10]:
from sklearn.ensemble import AdaBoostRegressor
clf_aBR = AdaBoostRegressor(random_state=51, n_estimators=1000)
rmsle_aBR = test_model(clf_aBR)
print (rmsle_aBR, test_model_r2(clf_aBR))

[1.4190287735125264] [-2.7424067554634184]


In [11]:
from sklearn.ensemble import BaggingRegressor
clf_bgr = BaggingRegressor(base_estimator=None, bootstrap=True, bootstrap_features=False,
                 max_features=1.0, max_samples=1.0, n_estimators=100,
                 n_jobs=None, oob_score=False, random_state=51, verbose=0,
                 warm_start=False)
rmsle_bgr = test_model(clf_bgr)
print (rmsle_bgr, test_model_r2(clf_bgr))

[0.4996648480065363] [0.3343879334064715]


In [12]:
import xgboost as xgb
from xgboost import plot_importance

clf_xgb = xgb.XGBRegressor()
rmsle_xgb = test_model(clf_xgb)
print (rmsle_xgb, test_model_r2(clf_xgb))

[0.5157659191028408] [0.31528494155822007]


In [13]:
from sklearn.ensemble import GradientBoostingRegressor
clf_gbr = GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='friedman_mse',
                          init=None, learning_rate=0.05, loss='ls', max_depth=3,
                          max_features='sqrt', max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=9, min_samples_split=8,
                          min_weight_fraction_leaf=0.0, n_estimators=1250,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=0.8, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

rmsle_ggr = test_model(clf_gbr)
print (rmsle_ggr, test_model_r2(clf_gbr))

[0.5363375605325416] [0.3291693431014995]


In [16]:
X = np.nan_to_num(test_dataset.drop(['Date'], axis=1))

clf_gbr.fit(x_train, y_train)
Y_pred_gbr = clf_gbr.predict(X) 

#Bagging
clf_bgr.fit(x_train, y_train)
Y_pred_bgr = clf_bgr.predict(X) 

#ridge

#Elastic
reg_elastic_net.fit(x_train, y_train)
Y_pred_elastic = reg_elastic_net.predict(X) 

C:\Users\Aboli\Anaconda3\envs\Intern1\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:476: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9631095.766126482, tolerance: 5575.570966499806
  positive)


In [22]:
Y_pred_gbr = Y_pred_gbr.astype(int)

Y_pred_bgr = Y_pred_bgr.astype(int)

Y_pred_elastic = Y_pred_elastic.astype(int)

In [23]:
y = pd.DataFrame({"WattHour":Y_pred_bgr})
y = y.round(-1)
y=y.head(4375)


In [24]:
y.to_csv("B1.csv",index=False)